In [14]:
import os
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

In [15]:
pip install -q -U google-genai

Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install python-dotenv


In [17]:
from dotenv import load_dotenv
load_dotenv()

import os
api_key = os.getenv("GEMINI_API_KEY")


In [18]:
pip install langchain langchain-google-genai


Note: you may need to restart the kernel to use updated packages.


In [19]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.messages import HumanMessage

llm = GoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7,
    google_api_key=api_key
)

response = llm.invoke([HumanMessage(content="What is NVIDIA?")])
print(response)

NVIDIA is a multinational technology company primarily known for its **graphics processing units (GPUs)**.  While initially focused on the gaming market, NVIDIA's GPUs have become crucial components in a wide range of industries, including:

* **Gaming:**  Powering high-performance gaming PCs and consoles.
* **Artificial Intelligence (AI):**  Used extensively for deep learning and machine learning tasks due to their parallel processing capabilities.
* **Data Centers:**  Providing the computing power for large-scale data processing and AI workloads in cloud computing environments.
* **Professional Visualization:**  Used in fields like CAD, digital content creation, and medical imaging.
* **Automotive:**  Powering advanced driver-assistance systems (ADAS) and autonomous vehicles.

Beyond GPUs, NVIDIA also develops and manufactures:

* **System-on-a-chip (SoC) units:**  Combining CPU, GPU, and other components onto a single chip.
* **Software:**  Developing software platforms and drivers 

In [20]:
loaders = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])
data = loaders.load() 
len(data)

2

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. Initialize the splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# 2. Split documents while preserving metadata
docs = text_splitter.split_documents(data)


In [34]:
len(docs)

17

In [23]:
pip install sentence-transformers


In [35]:
!pip install hf_xet
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [36]:
vectorindex= FAISS.from_documents(docs, embeddings)

In [37]:
# Storing vector index create in local
file_path="vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorindex, f)

In [38]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

retriever = vectorIndex.as_retriever()

In [44]:
# 6. Retrieval QA Chain using "stuff" instead of "map_reduce"

from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

# 7. Ask a question
query = "What is the price of Tiago iCNG?"
result = qa_chain.invoke({"query": query})

# 8. Print result
print("Answer:", result["result"])
print("Sources:")
for doc in result["source_documents"]:
    print("-", doc.metadata.get("source", "Unknown"))

Answer: The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh.

Sources:
- https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html
- https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html
- https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html
- https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html


- https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html
- https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html
- https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html
- https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html
